In [27]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Dataset

In [28]:
import torchvision.models as models
import torch.nn as nn
import os
from PIL import Image
import tqdm
import torch

class CatDogDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with subdirectories for each class.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_names = ['antelope', 'bear', 'bird', 'cat', 'dog']

        # Load all image file paths and their labels
        for class_index, sub_dir in enumerate(self.class_names):
            class_path = os.path.join(root_dir, sub_dir)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                        img_path = os.path.join(class_path, img_file)
                        self.samples.append((img_path, class_index))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


In [29]:
# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the dataset
dataset = CatDogDataset(root_dir='Classified_Animals', transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CatOrNotCNN(nn.Module):
    def __init__(self):
        super(CatOrNotCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=64 * 56 * 56, out_features=512)
        self.fc2 = nn.Linear(512, 5)  # 2 classes: cat or not

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [31]:
import matplotlib.pyplot as plt

def visualize_feature_maps(feature_maps):
    # Assuming feature_maps is a list of tensors from the convolutional layers
    for layer, f_map in enumerate(feature_maps):
        layer += 1  # Start layer indexing at 1
        plt.figure(figsize=(20, 15))
        channels = f_map.shape[1]
        
        for i in range(channels):
            plt.subplot(channels // 8 + 1, 8, i + 1)  # Arrange plots in a grid
            plt.imshow(f_map[0, i].detach().cpu().numpy(), cmap='gray')
            plt.axis('off')
        
        plt.show()


In [32]:
def train_model(model, train_loader, val_loader, num_epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    for epoch in tqdm.tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm.tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)  # No need for conv_outputs here
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)  # No need for conv_outputs here
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Accuracy: {100 * correct / total}%')

    print('Finished Training')

model = CatOrNotCNN()
train_model(model, train_loader, val_loader, num_epochs=5)

 20%|██        | 1/5 [00:15<01:03, 15.79s/it]

Epoch 1, Loss: 2.2953444520632424, Validation Loss: 1.4486250281333923, Accuracy: 39.583333333333336%


 40%|████      | 2/5 [00:30<00:45, 15.13s/it]

Epoch 2, Loss: 1.0146890183289845, Validation Loss: 1.6233793497085571, Accuracy: 41.666666666666664%


 60%|██████    | 3/5 [00:44<00:29, 14.78s/it]

Epoch 3, Loss: 0.7282686233520508, Validation Loss: 1.3493443727493286, Accuracy: 47.916666666666664%


 80%|████████  | 4/5 [00:59<00:14, 14.63s/it]

Epoch 4, Loss: 0.43446006377538043, Validation Loss: 1.3934924006462097, Accuracy: 52.083333333333336%


100%|██████████| 5/5 [01:14<00:00, 14.82s/it]

Epoch 5, Loss: 0.28306764860947925, Validation Loss: 1.2788781523704529, Accuracy: 60.416666666666664%
Finished Training


In [33]:
# Assume 'model' is your trained CatOrNotCNN model
# and 'val_loader' is your DataLoader for the validation set

# Turn model to evaluation mode
model.eval()

# Get a single batch of images
images, labels = next(iter(val_loader))

# Move images to the same device as model
images = images.to(next(model.parameters()).device)

# Forward pass to get outputs and feature maps
with torch.no_grad():
    _, feature_maps = model(images)

# Visualize feature maps for the first image in the batch
visualize_feature_maps(feature_maps)


ValueError: too many values to unpack (expected 2)